In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import AsinhStretch, ImageNormalize
import sys
sys.path.insert(0, '/Users/smericks/Desktop/StrongLensing/darkenergy-from-lagn/')
from lens_catalog import OM10LensCatalog

## Investigate a $v_{rms}$ map from SKiNN ##

In [ ]:
gt_lenses = OM10LensCatalog('../MassModels/om10_sample/om10_venkatraman_erickson24.csv')
om10_catalog = pd.read_csv('../MassModels/om10_sample/om10_venkatraman_erickson24.csv')

In [20]:
hst_images = np.load('../MassModels/om10_sample/hst_images.npy')

In [ ]:
vrms_maps = np.load('first10_vrms.npy')
vrms_maps_i85 = np.load('first10_vrms_i85.npy')

zoom_pix = 30

fig,axs = plt.subplots(2,5,dpi=200,figsize=(6,2))
image_norm = ImageNormalize(hst_images[3],stretch=AsinhStretch())
vrms_norm = ImageNormalize(vrms_maps[0])
for i in range(0,5):
    plt_im = axs[0,i%5].matshow(hst_images[i][zoom_pix:-zoom_pix,zoom_pix:-zoom_pix],norm=image_norm)
    plt.plot()
    min=60+(zoom_pix*2)
    plt_vrms = axs[1,i%5].matshow(vrms_maps[i][min:-min,min:-min],norm=vrms_norm)
    axs[0,i%5].set_xticks([])
    axs[0,i%5].set_yticks([])
    axs[1,i%5].set_xticks([])
    axs[1,i%5].set_yticks([])
    #if i == 4:
    #    cbar = fig.colorbar(plt_im,ax=axs[0,5])
    #    cbar = fig.colorbar(plt_vrms,ax=axs[1,5])
    #    axs[0, 5].axis('off')
    #    axs[1, 5].axis('off')
        #fig.colorbar(plt_im, ax=axs[0, 5], orientation='vertical')
        #fig.colorbar(plt_vrms, ax=axs[1, 5], orientation='vertical')
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.53, 0.015, 0.35])
fig.colorbar(plt_im, cax=cbar_ax)
cbar_ax = fig.add_axes([0.82, 0.11, 0.015, 0.35])
fig.colorbar(plt_vrms, cax=cbar_ax)
plt.suptitle('i=90')

"""
fig,axs = plt.subplots(2,5,dpi=200,figsize=(5,2))
for i in range(0,10):
    axs[i//5,i%5].matshow(vrms_maps_i85[i])
    axs[i//5,i%5].set_xticks([])
    axs[i//5,i%5].set_yticks([])
plt.suptitle('i=85')
"""